In [ ]:
from IPython.display import display, Math, Latex

import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import random


## Group Assignment
### Team Number: 3
### Team Member Names: Jeffrey Zhao, Bethany Liu, Ray Wang
### Team Strategy Chosen: Safe

Our team decided to aim for the **SAFEST** portfolio.

The overall thought process behind our solution combines different key topics and calculations that we've learned throughout this course. Strategies we thought of to obtain the safest portfolio include:

- **SMALLEST** standard deviation
- **SMALLEST** beta
- most **NEGATIVELY CORRELATED** stocks
- the **MOST** stocks as possible (22) (diversification)

These ideas will be seen and expanded upon throughout the rest of our code.
Here is a brief overview:


Blueprint of the code:
- Read in CSV File
- Get rid of tickers that don't meet requirements
- Calculate standard deviation
- Calculate beta
- Determine optimal portfolio by taking the 26 stock of smallest std + beta value
- Calculate correlation between each stock and get rid of 4 stocks with highest correlation
- Calculate optimal weightings on each stock
- Generate final portfolio
- Test portfolio

(Side note: Although it was discussed, our team did not decide to try and earn back our transaction fees. We decided that in the grand scheme of things, if we buy 20 stocks, the fee will only be $\$100$ which is extremely minimal compared to our $\$750000$ portfolio. ($\$100$ is approximately 0.013% of $\$750000$). Furthermore, we also tried looking at investing in different industries, however, .info is broken.)

In [ ]:
# Read in CSV file and filter out tickers that do not meet the requirements

# Requirements:
# - Stock must be denominated in either USD or CAD
# - Stocks must have an average monthly volume of at least 150,000 shares between Jan 1 - Oct 31, 2023
# - Stocks must have at least 18 trading days

# Read in CSV file
tickers_data = pd.read_csv('Tickers.csv', header=None) #dataframe for tickers file, change to 'Tickers' after we finish the code
tickers_data.set_index(tickers_data.columns[0], inplace=True)

# using these dates for data for now, can be changed later
start_date = '2023-01-01'
end_date = '2023-10-31'

# create function to get data of every stock
print("Please ignore error messages from yfinance")

def get_stock_data(ticker_df):
  complete_stock_data = pd.DataFrame()
  for i in ticker_df.index: # for loop to get every element of the dataframe
    ticker = yf.Ticker(i)
    stock_history = ticker.history(start=start_date, end=end_date)
    if stock_history.empty:
      continue
    currency = ticker.fast_info['currency']

    # ignore stocks that have monthly volume of less than 150000 and stocks that are not listed in cad or usd
    if (monthly_volume(stock_history)>150000 and (currency == 'CAD'or currency == 'USD')):
        complete_stock_data[i] = stock_history.Close.pct_change()

  complete_stock_data.index = complete_stock_data.index.date

  return complete_stock_data

# create function to get the average monthly volume from january 1st to october 31st, get rid of months that have less than 18 trading days
def monthly_volume(df):
  min_trading_days = 18
  trading_days_per_month = df.resample('M').size() # find number of trading days per month
  valid_months = trading_days_per_month[trading_days_per_month >= min_trading_days].index # check if the number of days is greater than 18

  # compute average volume
  total_volume = 0
  for month in valid_months:
     total_volume += df[df.index.month == month.month].Volume.mean()
  return total_volume/len(valid_months)

# main
complete_portfolio = get_stock_data(tickers_data)
print("Complete portfolio of all the stocks in the ticker file:", '\n')
complete_portfolio.head()

FileNotFoundError: ignored

**Calculating the Standard Deviation**

From the modules, we learned that the standard deviation of a stock is the calculation of a stocks volatility (the tendancy for the stock price to change). So in order to achieve the safest portfolio (least change), we want to have stocks with the lowest standard deviation. Therefore, we started off by calculating the standard deviation of the percentage returns of each stock in order to compare each of their volatilities.


In [ ]:
# create function to calculate the standard deviation
def standard_deviation(df):
  std = pd.DataFrame()
  std = df.std()
  return std

print("Standard deviation of the percentage returns for each stock in the portfolio")
display(standard_deviation(complete_portfolio))

**Calculating the Beta**

We also learned that the beta is also a measure of the volaility of a security's return. However, it measures it in comparison to the market. We decided to consider this since we want our stocks to be less correlated to the market, meaning that it would be less influenced by the fluctuations in the market. (For the market, we decided to use the S&P 500 as it would give the best indication to how the market is doing)

In [ ]:
# graph the closing prices of the market
start_date1 = '2023-01-01'
end_date1 = '2023-10-31'
investment = 750000

# import S&P 500
MarketIndex='^GSPC'
Ticker2 = yf.Ticker(MarketIndex)
MarketIndex_hist = Ticker2.history(start=start_date, end=end_date)
shares = investment/MarketIndex_hist['Close'].iloc[0]
Market_portfolio = MarketIndex_hist['Close']*shares
Marketdf = MarketIndex_hist['Close'].pct_change()
Marketdf.index = Marketdf.index.date

plt.figure(figsize = (13,6))
plt.plot(Market_portfolio, label = "S&P 500")
plt.title(f"Portfolio Values for the S&P 500 between {start_date1} and {end_date1}")
plt.xlabel("Date")
plt.ylabel("S&P 500 Portfolio Values (USD)")
plt.legend()
plt.show()

print("The standard deviation of the S&P 500 is", MarketIndex_hist['Close'].pct_change().std())
print(f"The capital gain of the S&P 500 between {start_date1} and {end_date1} is {abs(Market_portfolio.iloc[-1] - Market_portfolio.iloc[0])}")

From above, we can see that the market seems to change constantly between the beginning of this year. So, we decided that it is not optimal for the stocks we choose to be highly correlated with the market as it would then also be as volatile as the market.

In [ ]:
# calculate the beta
# takes in a dataframe with the close prices and outputs a dataframe of one row containing all the betas for each stock
'''
Beta 	 Meaning
1.0	The stock moves in line with the broader market
2.0	The stock moves twice as much as the broader market
0.0	The stock's moves don’t correlate with the broader market
-1.0	The stock moves in the opposite direction of the broader market
'''

def beta(df):
  #create dataframe for the beta values
  betadf = pd.DataFrame()
  for i in df.columns:
    #create a dataframe with a single stocks prices and the market index
    prices = pd.DataFrame()
    prices[i] = df[i]
    prices[MarketIndex] = Marketdf
    #calculate variance of market
    MarketVar = prices[MarketIndex].var()
    #drop first row since NA
    prices.drop(index=prices.index[0], inplace=True)
    #calculate beta which returns a 2x2 dataframe
    betacalc = prices.cov()/MarketVar
    #select the value for the beta of the selected stock
    betadf[i] = pd.Series(betacalc.iat[0,1])

  return betadf

print("The beta for each stock is:", '\n')
display(beta(complete_portfolio))

**Adding the Standard Deviation and Beta**

After calculating the standard deviation and beta of each stock, we decided to add the two values together to determine the extent of its volatility and compare it with the other stocks. We chose to priioritize the standard deviation of the stock more than its beta since the standard deviation is more of an indicator of a stock's volatility than its beta. Therefore, we chose a weighting of 70% on the standard deviation and 30% on the beta.

In [ ]:
# add standard deviation and beta

# market movement is not as impactful as the volatility of the stock itself so put weighting on each in relation to importance
std_weight = 0.7
beta_weight = 0.3

# function to calculate the sum of the standard deviation and beta
def add_std_and_beta(std, beta):
  std_and_beta = pd.DataFrame()
  std_and_beta = std*std_weight*100 + abs(beta)*beta_weight
  return std_and_beta

std_and_beta = add_std_and_beta(standard_deviation(complete_portfolio), beta(complete_portfolio))
display(std_and_beta)

After calculating the sum, we then took the top 24 stocks with the lowest standard deviation and beta to filter out stocks with the highest volatility.

In [ ]:
# get the top 24 stocks with least standard deviation and beta
# returns a dataframe with the list of 24 tickers from the least volatile to most volatile based off our calculations
portfolio = pd.DataFrame()
def get_portfolio(std_and_beta):
   std_and_beta = std_and_beta.iloc[0, :].sort_values()
   if len(std_and_beta) >= 22:
    portfolio = pd.DataFrame(std_and_beta.head(min(24, len(std_and_beta))))
   else:
    portfolio = std_and_beta


   return portfolio

# main
generated_portfolio = get_portfolio(std_and_beta)
generated_portfolio.rename(columns={0: 'Std and Beta value'}, inplace=True)
display(generated_portfolio)

**Calculating the Correlation**

Next, we calculated the correlation between each of our 24 chosen stocks. To obtain the safest portfolio, it is important to have a diversified portfolio, meaning that the stocks that we choose should have as minimal of a correlation as possible (optimal would be a perfect negative correlation, which means 0 capital gain/loss, however that is extremely difficult to obtain). After calculating the correlation between each stock, we get rid of 2 stocks (one in each pair) of the highest correlation in attempts to divsersify the industries of the different stocks (the higher the correlation, the more likely they are to be in the same industry).

In [ ]:
# calculate correlation between each chosen stock
correlation = get_stock_data(generated_portfolio).corr()

# filter out stocks with a correlation greater than 0.75 or 1 (same stock)
correlation = (correlation[(correlation<.75) | (correlation == 1)])
print("The beginning of the dataframe containing the correlations between the stocks is:")
display(correlation.head())

**Removing high correlation stocks**

We go through the portfolio in reverse by considering the columns of the correlation dataframe, If there is a NaN value in the column, it means this ticker has a high correlation with another ticker. We then compare the row indices for when there is NaN value, and if the row indices are still in the portfolio, we remove the column ticker from the portfolio. If the row indices do not appear in the portfolio, it means it has aleady been removed previously. In this case we do not remove the column because we still want to keep at least one stock from a pair that has high correlation

In [ ]:
columns_reverse = list(reversed(correlation.columns))
for i in columns_reverse:
  if correlation[i].isnull().any() and len(generated_portfolio)>=10: # if there is a correlation value that is higher than 0.75, than it would be filtered out as NaN in the table
    null_indices = correlation.loc[correlation[i].isnull()].index # find the row indices where the the correlation is higher than 0.75
    common_indices = generated_portfolio.index.intersection(null_indices) # find the row indices that are also in the portfolio
    if common_indices.size != 0: # if there's no intersection, it means the row index has already been removed from the portfolio, so no need to remove the column as well
      print('The removed ticker is', i)
      generated_portfolio = generated_portfolio.drop(i) # if common row indices is not empty, get rid of the ticker of the column

if len(generated_portfolio>22): # after eliminating the high correlation stocks, if the number of stocks we have is greater than the required amount of 22, just take only the first 22 stocks
  generated_portfolio = generated_portfolio.head(22)
print('The updated portfolio after removing high correlation stocks:')
display(generated_portfolio)

**Calculating the Optimal Weighting on Each Stock**

After removing the high correlation stocks, we are left with the stocks we want to keep in our portfolio. The way we are going to determine weighting is to linearly increase the weighting every stock starting from the most volatile one. We are giving the volatile stocks less weighting so that it has less of an influence on the portfolio. The formula to generate the weighting is determined by the arithmetic sum formula with the sum being 100 and the starting value being 100/(2*number of tickers)



In [ ]:
# determine weighting/percentage of each stock
# using arithmetic sequence sum formula Sn = n/2(2a+(n-1)d), calculate the difference in weighting between each stock
# to evenly space out weighting of every stock, with the most volatile ones being the least weighted, least volatile being most weighted
# choose starting value, a, to be the minimum weighted value, 100/(2*n), as stated by the rules
n=len(generated_portfolio) # number of stocks
S = 100 # weighting has to add up to 100%
a = 100/(2*n)
d = ((S*2/n)-2*a)/(n-1)
print("Difference in weighting between each stock:", d)
print("Weighting of the most volatile stock:", a)
print("Weighting of the least volatile stock:", a+21*d)

def create_final_portfolio(df):
  Portfolio_Final = pd.DataFrame(columns=['Ticker', 'Price', 'Currency', 'Shares', 'Values', 'Weight'], index=range(1, len(df)+1))
  start_date = '2023-11-21'
  end_date = '2023-11-21'
  usd_to_cad = yf.Ticker('USDCAD=x')
  exchange_rate = usd_to_cad.fast_info['previousClose']

  investment = 750000
  count = 1
  for i in df.index:
    investment = investment - 4.95 # subtract out the fees for the buying the stock

    ticker = yf.Ticker(i)
    stock_price = ticker.fast_info['previousClose'] # take the last close price of the stock
    currency = ticker.fast_info['currency']
    weight = a+(n-count)*d # calculates weighting of the stock

    Portfolio_Final.loc[count, 'Ticker'] = i
    Portfolio_Final.loc[count, 'Price'] = stock_price
    Portfolio_Final.loc[count, 'Currency'] = currency
    Portfolio_Final.loc[count, 'Weight'] = round(weight,2)
    if currency == 'USD':
     stock_price = stock_price*exchange_rate # turn stock price into CAD

    shares_values = investment*weight/100 # find value of this stock in CAD in the portfolio
    num_shares = shares_values/stock_price
    Portfolio_Final.loc[count, 'Shares'] = round(num_shares,2)
    Portfolio_Final.loc[count, 'Values'] = round(shares_values,2)

    count += 1

  columns_to_sum = Portfolio_Final[['Values','Weight']]
  Portfolio_Final.loc['Total'] = columns_to_sum.sum(axis=0)
  return Portfolio_Final
  #Ticker, Price, Currency, Shares, Value, Weight. Ticker will be the ticker your code selected, Price is the price on November 25, 2023,
  #Currency is either USD or CAD, Shares is the number of shares you purchased of that stock,
  #Value is the total value of those shares, and Weight is the weight that the value of shares represents relative to the value of your portfolio

final_portfolio = create_final_portfolio(generated_portfolio)

**Test Runs**

Now that we have our final portfolio, we need to test to see whether or not it works. We do this through building our portfolio by investing the determined weighting on each stock and adding the returns for of each invetment. We then compute the overall capital gain/loss and the standard deviation of the portfolio to compare the results.

Our test trials consist of the results of the most recent week, month and year to guage how well our code works.

In [ ]:
# Basic imports

# begining investment
investment = 750000 #(an estimation)

# import chosen tickers
test_tickers = final_portfolio['Ticker'].iloc[:-1].tolist()
print(test_tickers)

In [ ]:
# Test 1: 1 Week (2023-11-16 - 2023-11-23)

# change dates here
test1_start_date = '2023-11-25'
test1_end_date = '2023-12-01'

# get the closing prices of each ticker multiplied by the respective number of shares
def get_test_portfolio(test_tickers, start_date, end_date):
  closing = pd.DataFrame()
  count = 0

  for ticker in test_tickers:
    hist = yf.Ticker(ticker).history(start = start_date, end = end_date)
    shares = (final_portfolio['Weight'].iloc[count]*0.01*investment)/hist['Close'].iloc[0]
    closing[ticker] = hist['Close']*shares
    count += 1

  return closing

# main
test1_portfolio = get_test_portfolio(test_tickers, test1_start_date, test1_end_date)
test1_portfolio = test1_portfolio.dropna()
test1_portfolio.index = test1_portfolio.index.date
display(test1_portfolio.head())

# get the final diversified portfolio
final_test1_portfolio = test1_portfolio.sum(axis=1)
display(final_test1_portfolio.head())

def plot_portfolio(test_portfolio, start_date, end_date):
  # beginning value - ending value
  print("The capital gain/loss is,", abs(test_portfolio.iloc[-1] - test_portfolio.iloc[0]))
  print("The standard deviation of the final portfolio is", test_portfolio.pct_change().std())

  # plot graph
  plt.figure(figsize = (13,6))
  plt.plot(test_portfolio, label = "Test Portfolio")
  plt.title(f"Portfolio Values for the Test Tickers between {start_date} and {end_date}")
  plt.xlabel("Date")
  plt.ylabel("Test Portfolio Values (CAD)")
  plt.legend()
  plt.show()

# main
plot_portfolio(final_test1_portfolio, test1_start_date, test1_end_date)

In [ ]:
# Test 2: 1 Month (2023-10-01 - 2023-10-31)

test2_start_date = '2023-10-01'
test2_end_date = '2023-10-31'

# get portfolio
test2_portfolio = get_test_portfolio(test_tickers, test2_start_date, test2_end_date)
test2_portfolio = test2_portfolio.dropna()
test2_portfolio.index = test2_portfolio.index.date
display(test2_portfolio.head())

# get the final diversified portfolio
final_test2_portfolio = test2_portfolio.sum(axis=1)
display(final_test2_portfolio.head())

# plot graph
plot_portfolio(final_test2_portfolio, test2_start_date, test2_end_date)

In [ ]:
# Test 3: (approximately) 1 Year (2023-01-01 - 2023-11-24)

test3_start_date = '2023-01-01'
test3_end_date = '2023-10-31'

# get portfolio
test3_portfolio = get_test_portfolio(test_tickers, test3_start_date, test3_end_date)
test3_portfolio = test3_portfolio.dropna()
test3_portfolio.index = test3_portfolio.index.date
display(test3_portfolio.head())

# get the final diversified portfolio
final_test3_portfolio = test3_portfolio.sum(axis=1)

display(final_test3_portfolio.head())

# plot graph
plot_portfolio(final_test3_portfolio, test3_start_date, test3_end_date)

**Comparing our portoflio with the S&P 500**

In [ ]:

plt.figure(figsize = (13,6))
plt.plot(final_test3_portfolio.pct_change(), label = "Generated Portfolio")
plt.plot(Market_portfolio.pct_change(), label = "S&P 500")
plt.title(f"Generated Portfolio Percentage returns vs S&P 500 Percentage returns between {test3_start_date} and {test3_end_date}")
plt.xlabel("Date")
plt.ylabel("Percentage Returns")
plt.legend()
plt.show()

**Test Results**

From the 3 tests conducted above, it is clear to see that our portfolio does not reach a 0 return in any case. However, the standard deviation on all three are extremely low (Test 1: 0.002, Test 2: 0.008, Test 3: 0.006) indicating that our code is still works in finding a less volatile portfolio. Furthermore, when comparing the percentage returns of our portfolio with the percentage returns of the S&P 500, we can see that in general, our portfolio is less volatile.

**Finishing Touches**

After finalizing and testing the data required to create our safe portfolio, we can now output the final portfolio...

In [ ]:
display(final_portfolio)

... and convert it to csv.

In [ ]:
# output to csv file
Stocks_Final = final_portfolio[['Ticker', 'Shares']]
Stocks_Final = Stocks_Final.drop(index = 'Total')

display(Stocks_Final)

Stocks_Final.to_csv('Stocks_Group_03.csv', index = True)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Jeffrey Zhao, Bethany Liu, Ray Wang

**THE END**